In [1]:
import pandas as pd
import requests
import base64
from urllib.parse import urlencode
import json
import datetime

#import constants from KEYS module. create a module called KEYS.py and Add your own Client ID and Client Secret to it
import KEYS

In [2]:
songs_df = pd.read_csv('./data/artistAndTracks.csv', index_col=[0])

In [23]:
#for testing
songs_df = songs_df[:5]

In [3]:
songs_df

,artist,track
0,Black Sabbath,War Pigs
1,The Pack,This Shit Slappin'
2,M.I.A.,Paper Planes
3,Jimmy Smith,I'm Gonna Love You Just a Little More Babe
4,Dorrough,Ice Cream Paint Job
...,...,...
341,"Krave featuring Flo Rida, Pitbull, and Lil Jon",Go Crazy
342,T.I.,Rubberband Man
343,Rosalind Rice featuring French Montana,Hustler
344,The Edgar Winter Group,Frankenstein


In [4]:
#assign keys to values here

client_id = KEYS.CLIENT_ID
client_secret = KEYS.CLIENT_SECRET

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentiatls(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentiatls()
        return  {
            "Authorization" : f"Basic {client_creds_b64}",
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials",
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_headers = self.get_token_headers()
        token_data = self.get_token_data()
        r = requests.post(token_url, headers=token_headers, data=token_data)
        if r.status_code not in range(200,299):
            raise Exception("Could not Authenticate client")
            #return False
        data = r.json() #convert json to dict
        now = datetime.datetime.now() # create datetime obj for now
        access_token = data['access_token'] #retrieve token from response
        expires_in = data['expires_in'] #retrieve token lifetime
        print(expires_in)
        expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < datetime.datetime.now() #boolean to rep if expired
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers


    def get_resources(self, lookup_id, resource_type = 'albums', version = 'v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()


    def get_album(self, _id):
        return self.get_resources(_id, resource_type= 'albums')

    def get_artist(self, _id):
        return self.get_resources(_id, resource_type= 'artists')


    def base_search(self, query_params):
        headers = self.get_resource_header()
        
        endpoint = "https://api.spotify.com/v1/search"
        r = requests.get(endpoint, params=query_params, headers=headers)
         
        print(r.url)
        if r.status_code in range(200,299):
            return r.json()
        return {}

    def search(self, query=None, search_type = 'artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f'{k}:{v}' for k,v in query.items()])
        query_params = {
            "q":query,
            "type":search_type.lower()
        }
        
        return self.base_search(query_params)
    


In [6]:
client = SpotifyAPI(client_id, client_secret)

In [22]:
query = {
    "artist" : songs_df.loc[0,'artist'],
    "track" : songs_df.loc[0,'track']
}
tracks = client.search(query, search_type='track')

track_uri = tracks['tracks']['items'][0]['uri']

https://api.spotify.com/v1/search?q=artist%3ABlack+Sabbath+track%3AWar+Pigs&type=track


In [43]:
tracks['tracks']['items'][0]['uri']

'spotify:track:2rd9ETlulTbz6BYZcdvIE1'

In [12]:
songs_df['track_uri'] = ""

for idx, row in songs_df.iterrows():

    query = {
        "artist" : row['artist'],
        "track" : row['track']
    }
    tracks = client.search(query, search_type='track')
    
    try:
        track_uri = tracks['tracks']['items'][0]['uri']
        
    except(IndexError) as e:
        #update search params
        halfLen = len(row['track'])//2
        query.update({'track': row['track'][:halfLen]})
        del query['artist']
        #hit api again
        tracks = client.search(query, search_type='track')
        try:
            track_uri = tracks['tracks']['items'][0]['uri']
            
        except:
            
            query.update({'track':row['track'].split('"')[0]})
            
            tracks = client.search(query, search_type='track')
            try:
                track_uri = tracks['tracks']['items'][0]['uri']
            except:
                track_uri= ""
    
    songs_df.loc[idx, 'track_uri'] = track_uri

https://api.spotify.com/v1/search?q=artist%3ABlack+Sabbath+track%3AWar+Pigs&type=track
https://api.spotify.com/v1/search?q=artist%3AThe+Pack+track%3AThis+Shit+Slappin%27&type=track
https://api.spotify.com/v1/search?q=track%3AThis+Shit&type=track
https://api.spotify.com/v1/search?q=artist%3AM.I.A.+track%3APaper+Planes&type=track
https://api.spotify.com/v1/search?q=artist%3AJimmy+Smith+track%3AI%27m+Gonna+Love+You+Just+a+Little+More+Babe&type=track
https://api.spotify.com/v1/search?q=track%3AI%27m+Gonna+Love+You+Ju&type=track
https://api.spotify.com/v1/search?q=artist%3ADorrough+track%3AIce+Cream+Paint+Job&type=track
https://api.spotify.com/v1/search?q=artist%3AJ-Kwon+track%3ATipsy+%2709&type=track
https://api.spotify.com/v1/search?q=artist%3AN.W.A+track%3AAppetite+for+Destruction%22+%28portion+sampled+samples+%22Get+Me+Back+on+Time%2C+Engine+%239%22+by+Wilson+Pickett%29&type=track
https://api.spotify.com/v1/search?q=track%3AAppetite+for+Destruction%22+%28portion+sampled+samples+&type=tr

https://api.spotify.com/v1/search?q=artist%3ARye+Rye+featuring+M.I.A.+track%3ABang&type=track
https://api.spotify.com/v1/search?q=track%3ABa&type=track
https://api.spotify.com/v1/search?q=artist%3APitbull+track%3AHotel+Room+Service&type=track
https://api.spotify.com/v1/search?q=artist%3AThe+Who+track%3AWon%27t+Get+Fooled+Again&type=track
https://api.spotify.com/v1/search?q=artist%3AFrankie+Smith+track%3ADouble+Dutch+Bus&type=track
https://api.spotify.com/v1/search?q=artist%3AMissy+Elliott+featuring+Ludacris+track%3AGossip+Folks&type=track
https://api.spotify.com/v1/search?q=track%3AGossip&type=track
https://api.spotify.com/v1/search?q=artist%3AYoung+Jeezy+track%3ABottom+of+the+Map&type=track
https://api.spotify.com/v1/search?q=track%3ABottom+o&type=track
https://api.spotify.com/v1/search?q=artist%3AThe+Grass+Roots+track%3ALet%27s+Live+for+Today&type=track
https://api.spotify.com/v1/search?q=artist%3ARun-D.M.C.+track%3AIt%27s+Tricky&type=track
https://api.spotify.com/v1/search?q=artist%

https://api.spotify.com/v1/search?q=artist%3ARihanna+featuring+Young+Jeezy+track%3AHard&type=track
https://api.spotify.com/v1/search?q=track%3AHa&type=track
https://api.spotify.com/v1/search?q=artist%3AWale+featuring+Gucci+Mane+track%3APretty+Girls&type=track
https://api.spotify.com/v1/search?q=track%3APretty&type=track
https://api.spotify.com/v1/search?q=artist%3ADeftones+track%3AAround+the+Fur&type=track
https://api.spotify.com/v1/search?q=artist%3AFugazi+track%3AWaiting+Room&type=track
https://api.spotify.com/v1/search?q=artist%3ABoogie+Down+Productions+track%3ASouth+Bronx%22+%28portion+sampled+samples+%22Get+Up+Offa+That+Thing%22+by+James+Brown%29&type=track
https://api.spotify.com/v1/search?q=track%3ASouth+Bronx%22+%28portion+sampled+samples+%22&type=track
https://api.spotify.com/v1/search?q=track%3ASouth+Bronx&type=track
https://api.spotify.com/v1/search?q=artist%3AJay-Z+featuring+Swizz+Beatz+track%3AOn+to+the+Next+One&type=track
https://api.spotify.com/v1/search?q=track%3AOn+to+

https://api.spotify.com/v1/search?q=artist%3ABeastie+Boys+track%3ARoot+Down&type=track
https://api.spotify.com/v1/search?q=artist%3AWaka+Flocka+Flame+track%3AHard+in+da+Paint&type=track
https://api.spotify.com/v1/search?q=artist%3ACrime+Mob+track%3AKnuck+If+You+Buck&type=track
https://api.spotify.com/v1/search?q=artist%3AVan+Halen+track%3AEruption&type=track
https://api.spotify.com/v1/search?q=artist%3ADJ+Funk+track%3APop+Those+Thangs&type=track
https://api.spotify.com/v1/search?q=track%3APop+Thos&type=track
https://api.spotify.com/v1/search?q=artist%3AClipse+track%3AChampion&type=track
https://api.spotify.com/v1/search?q=artist%3AT-Pain+featuring+Young+Jeezy+track%3AReverse+Cowgirl&type=track
https://api.spotify.com/v1/search?q=track%3AReverse&type=track
https://api.spotify.com/v1/search?q=artist%3ASoulja+Boy+track%3ABird+Walk&type=track
https://api.spotify.com/v1/search?q=artist%3AParty+Boyz+featuring+Dorrough+and+Charlie+Boy+track%3AFlex+%28Remix%29&type=track
https://api.spotify.co

https://api.spotify.com/v1/search?q=artist%3AThe+Jackson+5+track%3AI+Want+You+Back&type=track
https://api.spotify.com/v1/search?q=artist%3ABeastie+Boys+track%3AIntergalactic%22+%28portion+sampled+samples+%22The+New+Style%22+by+Beastie+Boys%29&type=track
https://api.spotify.com/v1/search?q=track%3AIntergalactic%22+%28portion+sampled+samp&type=track
https://api.spotify.com/v1/search?q=track%3AIntergalactic&type=track
https://api.spotify.com/v1/search?q=artist%3ASimon+and+Garfunkel+track%3ACecilia&type=track
https://api.spotify.com/v1/search?q=artist%3AMain+Source+track%3ALooking+at+the+Front+Door&type=track
https://api.spotify.com/v1/search?q=artist%3AJames+Brown+track%3AFunky+Drummer&type=track
https://api.spotify.com/v1/search?q=artist%3ALudacris+featring+Lil+Scrappy+track%3AEverybody+Drunk%22+%28portion+sampled+samples+%22You+Don%27t+Want+Drama%22+by+8Ball+%26+MJG%29&type=track
https://api.spotify.com/v1/search?q=track%3AEverybody+Drunk%22+%28portion+sampled+sample&type=track
https://

https://api.spotify.com/v1/search?q=artist%3AKaty+Perry+featuring+Snoop+Dogg+track%3ACalifornia+Gurls&type=track
https://api.spotify.com/v1/search?q=track%3ACaliforn&type=track
https://api.spotify.com/v1/search?q=artist%3AFree+School+featuring+Kelis+and+apl.de.ap+track%3AGrey+Goose+%28Whatcha+Sippin%27+On%29&type=track
https://api.spotify.com/v1/search?q=track%3AGrey+Goose+%28Wha&type=track
https://api.spotify.com/v1/search?q=artist%3ABelinda+Carlisle+track%3AHeaven+Is+a+Place+on+Earth&type=track
https://api.spotify.com/v1/search?q=artist%3AJanet+Jackson+track%3ASomeone+to+Call+My+Lover&type=track
https://api.spotify.com/v1/search?q=artist%3AMSTRKRFT+featuring+N.O.R.E.+track%3ABounce&type=track
https://api.spotify.com/v1/search?q=track%3ABou&type=track
https://api.spotify.com/v1/search?q=artist%3AShorty+Long+track%3AFunction+at+the+Junction&type=track
https://api.spotify.com/v1/search?q=artist%3ADuke+Williams+and+the+Extremes+track%3AChinese+Chicken&type=track
https://api.spotify.com/v

In [13]:
songs_df

,artist,track,track_uri
0,Black Sabbath,War Pigs,spotify:track:2rd9ETlulTbz6BYZcdvIE1
1,The Pack,This Shit Slappin',spotify:track:4IeUAegXVbETb7FpgNDnSM
2,M.I.A.,Paper Planes,spotify:track:1ixbwbeBi5ufN4noUKmW5a
3,Jimmy Smith,I'm Gonna Love You Just a Little More Babe,spotify:track:1uu0es5Bu7orSWkqS5rXp7
4,Dorrough,Ice Cream Paint Job,spotify:track:57QFOU3f7O0gDLadxv5z8w
...,...,...,...
341,"Krave featuring Flo Rida, Pitbull, and Lil Jon",Go Crazy,spotify:track:2Y0wPrPQBrGhoLn14xRYCG
342,T.I.,Rubberband Man,spotify:track:6jN49cPLhJxsIUaoJ0yroP
343,Rosalind Rice featuring French Montana,Hustler,spotify:track:5ErtQGRKEibK2WHaA729O8
344,The Edgar Winter Group,Frankenstein,spotify:track:2l0Z9fY1E8Bep8opmhmDhq


In [14]:
songs_df[songs_df['track_uri'] == ""]

,artist,track,track_uri
144,Fabolous,Young'n (Holla Back),


## Collected all spotify trackuris except for one

In [16]:
songs_df.to_csv('./data/tracks_uri.csv')